In [2]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')



# terus ini nama file yang kita coba buat latihan
df=pd.read_csv('/content/gdrive/My Drive/SNI/Deli - 15-10-2023.csv')

df = df.loc[:,['created_at','entry_id','field1','field2','field3','field4','field5','field6','field7']]

# HILANGIN TULISAN T DAN Z YANG ADA
df['created_at']=df['created_at'].str.replace("T"," ")
df['created_at']=df['created_at'].str.replace("Z"," ")

df.rename(columns={'field1': 'Temperatur', 'field2': 'Kelembapan Relatif','field3': 'PM2.5', 'field4' : 'CO2', 'field5' : 'Tekanan Udara','field6' : 'Temperatur SHT31','field7' : 'Humidity SHT31'}, inplace=True)
df = df.dropna()
#change to local time +7
df['created_at'] = pd.to_datetime(df['created_at']).dt.tz_convert('Asia/Jakarta')


# hilangkan nilai co2 dan pm2.5 yang bernilai <= 0
df=df[(df['CO2']>0)]
df=df[(df['PM2.5']>0)]

#
df

Mounted at /content/gdrive


<ipython-input-2-82a59a72e497>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_at'] = pd.to_datetime(df['created_at']).dt.tz_convert('Asia/Jakarta')


,created_at,entry_id,Temperatur,Kelembapan Relatif,PM2.5,CO2,Tekanan Udara,Temperatur SHT31,Humidity SHT31
8485,2023-03-02 14:44:29+07:00,8486,0.0,0.0,2350,93764.20313,0.00000,5.0,0.00
8486,2023-03-02 14:46:31+07:00,8487,0.0,0.0,2350,93764.20313,0.00000,5.0,0.00
8487,2023-03-02 14:48:32+07:00,8488,0.0,23.0,2000,93769.11719,0.00000,3.0,0.00
9280,2023-03-04 13:11:10+07:00,9281,25.2,78.1,11,993.75000,93846.89063,0.0,4.19
9281,2023-03-04 13:13:10+07:00,9282,25.2,78.1,11,993.75000,93846.89063,0.0,4.19
...,...,...,...,...,...,...,...,...,...
158399,2023-10-12 04:34:24+07:00,158400,25.1,63.4,106,1115.63000,93855.92188,0.0,4.48
158400,2023-10-12 04:36:25+07:00,158401,25.1,63.4,106,1115.63000,93855.92188,0.0,4.48
158401,2023-10-12 04:38:25+07:00,158402,25.1,63.5,98,781.25000,93856.71875,0.0,4.37
158402,2023-10-12 04:40:25+07:00,158403,25.1,63.5,98,781.25000,93856.71875,0.0,4.37


In [3]:
# Ambil Kelembapan Relatif & PM 2.5 (KITA TAMBAHKAN INDEX BULAN, HARI DAN JAM) UNTUK DISORTING NANTI
df1 = df.loc[:,['created_at','entry_id','Temperatur','Kelembapan Relatif','PM2.5','CO2']]
df1['Bulan'] = pd.DatetimeIndex(df['created_at']).month
df1.set_index = df1['Bulan'].inplace=True
df1['Hari'] = pd.DatetimeIndex(df['created_at']).day
df1.set_index = df1['Hari'].inplace=True
df1['Jam'] = pd.DatetimeIndex(df['created_at']).hour
df1.set_index = df1['Jam'].inplace=True

df1

,created_at,entry_id,Temperatur,Kelembapan Relatif,PM2.5,CO2,Bulan,Hari,Jam
8485,2023-03-02 14:44:29+07:00,8486,0.0,0.0,2350,93764.20313,3,2,14
8486,2023-03-02 14:46:31+07:00,8487,0.0,0.0,2350,93764.20313,3,2,14
8487,2023-03-02 14:48:32+07:00,8488,0.0,23.0,2000,93769.11719,3,2,14
9280,2023-03-04 13:11:10+07:00,9281,25.2,78.1,11,993.75000,3,4,13
9281,2023-03-04 13:13:10+07:00,9282,25.2,78.1,11,993.75000,3,4,13
...,...,...,...,...,...,...,...,...,...
158399,2023-10-12 04:34:24+07:00,158400,25.1,63.4,106,1115.63000,10,12,4
158400,2023-10-12 04:36:25+07:00,158401,25.1,63.4,106,1115.63000,10,12,4
158401,2023-10-12 04:38:25+07:00,158402,25.1,63.5,98,781.25000,10,12,4
158402,2023-10-12 04:40:25+07:00,158403,25.1,63.5,98,781.25000,10,12,4


In [4]:
#Validasi CO2


# terus kita filter data nya per 2 menit(sesuai ama waktu register data thingspeak)
dfco2 = df1.groupby(pd.Grouper(key='created_at',freq='2min')).mean().reset_index()

#Hapus nilai CO2 yang di luar batas range
dfco2 = dfco2[(dfco2['CO2']>400) &  (dfco2['CO2'] <5000)]

# kita ambil nilai index nya sesuai yang ada di petik '',
dfco2 = dfco2.loc[:,['created_at','Temperatur','Kelembapan Relatif','CO2','Bulan','Hari','Jam']]

# dropna() fungsinya buat drop data yang kosong
dfco2 = dfco2.dropna()

# astype int ngubah angka ke int
dfco2['CO2']=dfco2['CO2'].round().astype(int)
dfco2['Bulan']=dfco2['Bulan'].round().astype(int)
dfco2['Hari']=dfco2['Hari'].round().astype(int)
dfco2['Jam']=dfco2['Jam'].round().astype(int)

#round(2) gunanya buat ngatur 2 angka dibelakang koma, bisa atur angka apa aja
dfco2

,created_at,Temperatur,Kelembapan Relatif,CO2,Bulan,Hari,Jam
1393,2023-03-04 13:10:00+07:00,25.2,78.1,994,3,4,13
1394,2023-03-04 13:12:00+07:00,25.2,78.1,994,3,4,13
1395,2023-03-04 13:14:00+07:00,25.4,76.6,947,3,4,13
1396,2023-03-04 13:16:00+07:00,25.4,76.6,947,3,4,13
1397,2023-03-04 13:18:00+07:00,25.6,76.9,703,3,4,13
...,...,...,...,...,...,...,...
160975,2023-10-12 04:34:00+07:00,25.1,63.4,1116,10,12,4
160976,2023-10-12 04:36:00+07:00,25.1,63.4,1116,10,12,4
160977,2023-10-12 04:38:00+07:00,25.1,63.5,781,10,12,4
160978,2023-10-12 04:40:00+07:00,25.1,63.5,781,10,12,4


In [8]:
#Outlier windowing per jam CO2
dfoutlier = pd.DataFrame(
    {
        "created_at": ["1990-10-09 00:00:00"],
        "Temperatur": ["0"],
        "Kelembapan Relatif": ["0"],
        "CO2": ["0"],
    },
    index=[0, 1, 2, 3],
)

x=1
while x<=10 :
  y=1
  while y<=31 :
    z=0
    while z<=23 :
      dfvalidasi = dfco2[(dfco2['Bulan']==x) & (dfco2['Hari']==y) & (dfco2['Jam']==z)]
      dfvalidasi = dfvalidasi.loc[:,['created_at','Temperatur','Kelembapan Relatif','CO2']]
      data = len(dfvalidasi.axes[0])
      jumlah = str(data)
      if (dfvalidasi.empty==True):
        pass
      else :
        Q3 = dfvalidasi['CO2'].describe()['75%']
        Q1 = dfvalidasi['CO2'].describe()['25%']
        IQR = Q3 - Q1
        UF = Q3 + (1.5*IQR)
        LF = Q1 - (1.5*IQR)
        dfvalidasi = dfvalidasi[(dfvalidasi['CO2'] < UF) & (dfvalidasi['CO2'] > LF)]

        dfoutlier = pd.concat([dfoutlier, dfvalidasi], axis=0, ignore_index=True)
      z+=1
    y+=1
  x+=1

dfoutlier['CO2'] = dfoutlier['CO2'].astype(int)
dfoutlier = dfoutlier[(dfoutlier['CO2']>400) &  (dfoutlier['CO2'] <5000)]

dfoutlier

,created_at,Temperatur,Kelembapan Relatif,CO2
4,2023-03-04 13:10:00+07:00,25.2,78.1,994
5,2023-03-04 13:12:00+07:00,25.2,78.1,994
6,2023-03-04 13:14:00+07:00,25.4,76.6,947
7,2023-03-04 13:16:00+07:00,25.4,76.6,947
8,2023-03-04 13:18:00+07:00,25.6,76.9,703
...,...,...,...,...
125173,2023-10-12 04:34:00+07:00,25.1,63.4,1116
125174,2023-10-12 04:36:00+07:00,25.1,63.4,1116
125175,2023-10-12 04:38:00+07:00,25.1,63.5,781
125176,2023-10-12 04:40:00+07:00,25.1,63.5,781


In [10]:
dfcount = dfoutlier
dfstd = dfoutlier

# ambil 1 parameter hanya untuk menghitung ketersediaan data
dfcount = dfcount.groupby(pd.Grouper(key='created_at',freq='1H')).count()[lambda x: x > 22].dropna().round().astype(int)
dfcount = dfcount['CO2'].reset_index()
dfcount = dfcount.rename(columns={'CO2':'(n)Data'})

# hitung std CO2
dfstd = dfstd.groupby(pd.Grouper(key='created_at',freq='1H')).std()
dfstd = dfstd['CO2'].reset_index().round(2)
dfstd = dfstd.rename(columns={'CO2':'σ (ppm)'})

dfdata = pd.merge(dfcount, dfstd, on='created_at', how='inner')

dfdata



<ipython-input-10-cdd36160b313>:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dfstd = dfstd.groupby(pd.Grouper(key='created_at',freq='1H')).std()


,created_at,(n)Data,σ (ppm)
0,2023-03-04 13:00:00+07:00,23,177.34
1,2023-03-04 14:00:00+07:00,29,186.12
2,2023-03-04 17:00:00+07:00,28,162.48
3,2023-03-04 18:00:00+07:00,30,144.73
4,2023-03-04 19:00:00+07:00,30,145.67
...,...,...,...
4175,2023-10-11 23:00:00+07:00,30,192.97
4176,2023-10-12 00:00:00+07:00,29,188.45
4177,2023-10-12 01:00:00+07:00,25,207.47
4178,2023-10-12 02:00:00+07:00,30,178.63


In [ ]:
# Gabungkan data per jam hanya dengan data yang tersedia 75%
df1hco2 = dfoutlier
df1hco2 = df1hco2.groupby(pd.Grouper(key='created_at',freq='1H')).mean().round(2)

df1hco2 = pd.merge(dfdata, df1hco2, on='created_at', how='inner')

df1hco2['CV (%)'] = ((df1hco2['σ (ppm)'] / df1hco2['CO2']) * 100).round(2)

df1hco2 = df1hco2.loc[:,['created_at','(n)Data','σ (ppm)','CV (%)','Temperatur','Kelembapan Relatif','CO2','Bulan','Hari']]


df1hco2

,created_at,(n)Data,σ (ppm),CV (%),Temperatur,Kelembapan Relatif,CO2,Bulan,Hari
0,2023-03-04 13:00:00+07:00,23,177.34,20.05,25.55,76.63,884.30,3.0,4.0
1,2023-03-04 14:00:00+07:00,29,186.12,19.54,25.84,77.56,952.52,3.0,4.0
2,2023-03-04 16:00:00+07:00,24,155.78,23.95,32.50,64.35,650.33,3.0,4.0
3,2023-03-04 17:00:00+07:00,28,162.48,24.24,30.41,68.06,670.18,3.0,4.0
4,2023-03-04 18:00:00+07:00,30,144.73,21.00,28.08,75.08,689.13,3.0,4.0
...,...,...,...,...,...,...,...,...,...
4204,2023-10-11 23:00:00+07:00,30,192.97,19.87,26.93,67.68,970.93,10.0,11.0
4205,2023-10-12 00:00:00+07:00,29,188.45,19.36,26.43,65.97,973.31,10.0,12.0
4206,2023-10-12 01:00:00+07:00,25,207.47,22.33,26.44,64.68,928.96,10.0,12.0
4207,2023-10-12 02:00:00+07:00,30,178.63,19.25,26.34,63.30,927.97,10.0,12.0


In [ ]:
path = '/content/gdrive/My Drive/SNI/Deli/CO2/Hari/'


x=1
while x<=12 :
  y=1
  while y<=31:
    bul = str(x)
    har = str(y)
    dfhari = df1hco2[(df1hco2['Bulan']==x) & (df1hco2['Hari']==y) ]

    data = len(dfhari.axes[0])
    jumlah = str(data)
    if (dfhari.empty==True):
      pass
    else :
      std = dfhari['CO2'].std(numeric_only=True).round(2)
      mean = dfhari['CO2'].mean()
      cv = ((std / mean)*100).round(2)
      sd = str(std)
      koef = str(cv)
      dfhari.to_csv(path+"Bulan "+bul+" Hari "+har+" | "+"SD: "+sd+" ppm "+ "CV: " +koef+"% | "+"CO2 Validation.csv", index=None)
    y+=1
  x+=1

In [ ]:
# 1H in Months
path = '/content/gdrive/My Drive/SNI/Deli/CO2/Bulan/'

x=1
while x<=12 :

  bul = str(x)
  dfbulan = df1hco2[(df1hco2['Bulan']==x)]
  data = len(dfbulan.axes[0])
  jumlah = str(data)
  if (dfbulan.empty==True):
    pass
  else :
    std = dfbulan['CO2'].std(numeric_only=True).round(2)
    mean = dfbulan['CO2'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    dfbulan.to_csv(path+"Bulan "+bul+ " | "+"SD: "+sd+" ppm "+ "CV: " +koef+"% | "+"CO2 Validation 1H.csv", index=None)
  x+=1

In [ ]:
df8hco2 = df1hco2
df8hco2 = df8hco2.groupby(pd.Grouper(key='created_at',freq='8H')).mean().round(2).reset_index()
df8hco2 = df8hco2.loc[:,['created_at','σ (ppm)','CV (%)','Temperatur','Kelembapan Relatif','CO2','Bulan','Hari']]



In [ ]:
df8hco2

,created_at,σ (ppm),CV (%),Temperatur,Kelembapan Relatif,CO2,Bulan,Hari
0,2023-03-04 08:00:00+07:00,181.73,19.80,25.70,77.10,918.41,3.0,4.0
1,2023-03-04 16:00:00+07:00,150.11,20.11,27.88,73.79,755.48,3.0,4.0
2,2023-03-05 00:00:00+07:00,157.98,18.38,25.59,77.34,868.37,3.0,5.0
3,2023-03-05 08:00:00+07:00,166.77,26.39,33.39,57.61,643.61,3.0,5.0
4,2023-03-05 16:00:00+07:00,176.31,22.06,27.96,70.28,813.81,3.0,5.0
...,...,...,...,...,...,...,...,...
661,2023-10-10 16:00:00+07:00,193.04,20.26,28.44,67.04,954.56,10.0,10.0
662,2023-10-11 00:00:00+07:00,186.45,19.27,27.65,62.73,975.00,10.0,11.0
663,2023-10-11 08:00:00+07:00,195.08,27.79,38.32,27.72,703.00,10.0,11.0
664,2023-10-11 16:00:00+07:00,187.51,22.13,30.45,55.86,848.21,10.0,11.0


In [ ]:
# 8H in Months
path = '/content/gdrive/My Drive/SNI/Deli/CO2/Bulan/'

x=1
while x<=12 :

  bul = str(x)
  dfbulan = df8hco2[(df8hco2['Bulan']==x)]
  data = len(dfbulan.axes[0])
  jumlah = str(data)
  if (dfbulan.empty==True):
    pass
  else :
    std = dfbulan['CO2'].std(numeric_only=True).round(2)
    mean = dfbulan['CO2'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    dfbulan.to_csv(path+"Bulan "+bul+ " | "+"SD: "+sd+" ppm "+ "CV: " +koef+"% | "+"CO2 Validation 8H.csv", index=None)
  x+=1

Nah, disini tuh seharusnya kita validasi langsung aja per jam dulu cari outlier, terus kalau data per jamnya kurang dari 22.5 (75% perjam) di drop di jam itu, kalau udah, baru kita sorting.

# PM2.5

In [ ]:
#Validasi PM2.5


dfpm = df1.groupby(pd.Grouper(key='created_at',freq='2min')).mean().reset_index()

#Hapus nilai PM2.5 yang di luar batas range
dfpm = dfpm[(dfpm['PM2.5']>0) &  (dfpm['PM2.5'] <250)]

#Correction factor for RH >= 80
dfpm.loc[dfpm['Kelembapan Relatif']>=80, ['PM2.5']] *= 0.67

dfpm = dfpm.loc[:,['created_at','Temperatur','Kelembapan Relatif','PM2.5','Bulan','Hari','Jam']]

dfpm = dfpm.dropna()

# astype int ngubah angka ke int
dfpm['PM2.5']=dfpm['PM2.5'].round().astype(int)
dfpm['Bulan']=dfpm['Bulan'].round().astype(int)
dfpm['Hari']=dfpm['Hari'].round().astype(int)
dfpm['Jam']=dfpm['Jam'].round().astype(int)

dfpm

,created_at,Temperatur,Kelembapan Relatif,PM2.5,Bulan,Hari,Jam
1393,2023-03-04 13:10:00+07:00,25.2,78.1,11,3,4,13
1394,2023-03-04 13:12:00+07:00,25.2,78.1,11,3,4,13
1395,2023-03-04 13:14:00+07:00,25.4,76.6,12,3,4,13
1396,2023-03-04 13:16:00+07:00,25.4,76.6,12,3,4,13
1397,2023-03-04 13:18:00+07:00,25.6,76.9,12,3,4,13
...,...,...,...,...,...,...,...
160975,2023-10-12 04:34:00+07:00,25.1,63.4,106,10,12,4
160976,2023-10-12 04:36:00+07:00,25.1,63.4,106,10,12,4
160977,2023-10-12 04:38:00+07:00,25.1,63.5,98,10,12,4
160978,2023-10-12 04:40:00+07:00,25.1,63.5,98,10,12,4


In [ ]:
dfcount = dfpm
dfstd = dfpm


dfcount = dfcount.groupby(pd.Grouper(key='created_at',freq='1H')).count()[lambda x: x > 22].dropna().round().astype(int)
# ambil 1 parameter hanya untuk menghitung ketersediaan data
dfcount = dfcount['PM2.5'].reset_index()
dfcount = dfcount.rename(columns={'PM2.5':'(n)Data'})

# hitung std PM2.5
dfstd = dfstd.groupby(pd.Grouper(key='created_at',freq='1H')).std()
dfstd = dfstd['PM2.5'].reset_index().round(2)
dfstd = dfstd.rename(columns={'PM2.5':'σ (µgr/m3)'})

dfdata = pd.merge(dfcount, dfstd, on='created_at', how='inner')



# ambil 1 parameter

In [ ]:
# Gabungkan data per jam hanya dengan data yang tersedia 75%
df1hpm = dfpm
df1hpm = df1hpm.groupby(pd.Grouper(key='created_at',freq='1H')).mean().round(2)

df1hpm = pd.merge(dfdata, df1hpm, on='created_at', how='inner')

df1hpm['CV (%)'] = ((df1hpm['σ (µgr/m3)'] / df1hpm['PM2.5']) * 100).round(2)

df1hpm = df1hpm.loc[:,['created_at','(n)Data','σ (µgr/m3)','CV (%)','Temperatur','Kelembapan Relatif','PM2.5','Bulan','Hari']]


df1hpm

,created_at,(n)Data,σ (µgr/m3),CV (%),Temperatur,Kelembapan Relatif,PM2.5,Bulan,Hari
0,2023-03-04 13:00:00+07:00,23,1.08,9.74,25.55,76.63,11.09,3.0,4.0
1,2023-03-04 14:00:00+07:00,29,3.75,23.13,25.84,77.56,16.21,3.0,4.0
2,2023-03-04 16:00:00+07:00,30,3.98,29.42,32.67,64.00,13.53,3.0,4.0
3,2023-03-04 17:00:00+07:00,28,1.61,9.03,30.41,68.06,17.82,3.0,4.0
4,2023-03-04 18:00:00+07:00,30,4.07,16.41,28.08,75.08,24.80,3.0,4.0
...,...,...,...,...,...,...,...,...,...
4183,2023-10-11 23:00:00+07:00,30,0.00,0.00,26.93,67.68,93.00,10.0,11.0
4184,2023-10-12 00:00:00+07:00,29,0.00,0.00,26.43,65.97,93.00,10.0,12.0
4185,2023-10-12 01:00:00+07:00,25,38.62,24.56,26.44,64.68,157.24,10.0,12.0
4186,2023-10-12 02:00:00+07:00,30,10.22,7.30,26.34,63.30,140.03,10.0,12.0


In [ ]:
df1hpm

,created_at,(n)Data,σ (µgr/m3),CV (%),Temperatur,Kelembapan Relatif,PM2.5,Bulan,Hari
0,2023-03-04 13:00:00+07:00,23,1.08,9.74,25.55,76.63,11.09,3.0,4.0
1,2023-03-04 14:00:00+07:00,29,3.75,23.13,25.84,77.56,16.21,3.0,4.0
2,2023-03-04 16:00:00+07:00,30,3.98,29.42,32.67,64.00,13.53,3.0,4.0
3,2023-03-04 17:00:00+07:00,28,1.61,9.03,30.41,68.06,17.82,3.0,4.0
4,2023-03-04 18:00:00+07:00,30,4.07,16.41,28.08,75.08,24.80,3.0,4.0
...,...,...,...,...,...,...,...,...,...
4183,2023-10-11 23:00:00+07:00,30,0.00,0.00,26.93,67.68,93.00,10.0,11.0
4184,2023-10-12 00:00:00+07:00,29,0.00,0.00,26.43,65.97,93.00,10.0,12.0
4185,2023-10-12 01:00:00+07:00,25,38.62,24.56,26.44,64.68,157.24,10.0,12.0
4186,2023-10-12 02:00:00+07:00,30,10.22,7.30,26.34,63.30,140.03,10.0,12.0


In [ ]:
path = '/content/gdrive/My Drive/SNI/Deli/PM2.5/Hari/'


x=1
while x<=12 :
  y=1
  while y<=31:
    bul = str(x)
    har = str(y)
    dfhari = df1hpm[(df1hpm['Bulan']==x) & (df1hpm['Hari']==y) ]

    data = len(dfhari.axes[0])
    jumlah = str(data)
    if (dfhari.empty==True):
      pass
    else :
      std = dfhari['PM2.5'].std(numeric_only=True).round(2)
      mean = dfhari['PM2.5'].mean()
      cv = ((std / mean)*100).round(2)
      sd = str(std)
      koef = str(cv)
      dfhari.to_csv(path+"Bulan "+bul+" Hari "+har+" | "+"SD: "+sd+" µgr.m3^-1 "+ "CV: " +koef+"% | "+"PM2.5 Validation.csv", index=None)
    y+=1
  x+=1

In [ ]:
# 1H in Months
path = '/content/gdrive/My Drive/SNI/Deli/PM2.5/Bulan/'

x=1
while x<=12 :

  bul = str(x)
  dfbulan = df1hpm[(df1hpm['Bulan']==x)]
  data = len(dfbulan.axes[0])
  jumlah = str(data)
  if (dfbulan.empty==True):
    pass
  else :
    std = dfbulan['PM2.5'].std(numeric_only=True).round(2)
    mean = dfbulan['PM2.5'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    dfbulan.to_csv(path+"Bulan "+bul+ " | "+"SD: "+sd+" µgr.m3^-1 "+ "CV: " +koef+"% | "+"PM2.5 Validation 1H.csv", index=None)
  x+=1

In [ ]:
df8hpm = df1hpm
df8hpm = df8hpm.groupby(pd.Grouper(key='created_at',freq='8H')).mean().round(2).reset_index()
df8hpm = df8hpm.loc[:,['created_at','σ (µgr/m3)','CV (%)','Temperatur','Kelembapan Relatif','PM2.5','Bulan','Hari']]



In [ ]:
# 8H in Months
path = '/content/gdrive/My Drive/SNI/Deli/PM2.5/Bulan/'

x=1
while x<=12 :

  bul = str(x)
  dfbulan = df8hpm[(df8hpm['Bulan']==x)]
  data = len(dfbulan.axes[0])
  jumlah = str(data)
  if (dfbulan.empty==True):
    pass
  else :
    std = dfbulan['PM2.5'].std(numeric_only=True).round(2)
    mean = dfbulan['PM2.5'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    dfbulan.to_csv(path+"Bulan "+bul+ " | "+"SD: "+sd+" µgr.m3^-1 "+ "CV: " +koef+"% | "+"PM2.5 Validation 8H.csv", index=None)
  x+=1